This code is to classify the selected examples automatically for those criteria that allow automatic classification.

In [ ]:
import pandas as pd
import xlrd
import re
from re import search

In [ ]:
# opens the excel sheet and reads it
# excel sheet needs to be in the same working directory as this file
# requires installation of xlrd >= 1.0.0 for Excel support (conversion to .xlsx required by openpyxl)

file = 'selected_rows_sorted.xls'
data = pd.ExcelFile(file)

 # returns the all the sheet names within the excel file
print(data.sheet_names)

In [ ]:
# parses the sheet into a data frame to show the column structure within the file

data_frame = data.parse('Sheet1')
data_frame.info

# only shows the first 10 rows
data_frame.head(10)

In [ ]:
# reads in the spreadsheet data

workbook = xlrd.open_workbook('selected_rows_sorted.xls')
sheet = workbook.sheet_by_name('Sheet1')

# gets the first sheet
sheet_1 = workbook.sheet_by_index(0)

row_count = sheet.nrows
col_count = sheet.ncols
print(f'Total rows: {row_count}\nTotal columns: {col_count}')

We're now going to create a new list of lists out of the file, and populate the empty columns with as much data as possible.

The data that should be easy to classify automatically are: est-ce que / wh- at the beginning or end of sentence / presence of c'est.

In [ ]:
# initialises list of lists
all_rows = []

for rx in range(sheet.nrows): # rx is an int, row is a list 
    row = sheet.row(rx)

    # initialises list that will contain our data
    data_list = []

    # populates list that stores all rows
    data_list.append(row[0].value)
    data_list.append(row[1].value)
    data_list.append(row[2].value) # wh
    data_list.append(row[3].value) # int
    data_list.append(row[4].value) # other
    data_list.append(row[5].value) # segment
    data_list.append(row[6].value)
    data_list.append(row[7].value)
    data_list.append(row[8].value)
    data_list.append(row[9].value)
    data_list.append(row[10].value)
    data_list.append(row[11].value)
    data_list.append(row[12].value)

    all_rows.append(data_list)
    # print(all_rows)
    
# prints extracted rows
print(f'We created a list that embeds {len(all_rows)} lists.')

In [ ]:
# iterates through list of lists and populates empty cells

wh = 'comment'
string = "est-ce"
string_count = 0
string2 = 'c\'est'
string2_count = 0

ex_situ_count = 0
in_situ_count = 0
fragment_count = 0
sv_count = 0

for list in all_rows:
    text = list[5] # index 5 is where text of interest is stored, returns a cell

    if search(string, text):
        list[2] = 'ex situ'
        list[3] = 'est-ce que'
        string_count += 1
    elif search(string2, text):
        list[3] = 'cleft'
        string2_count += 1

    if re.search('^comment\s', text, flags=0):
        list[2] = 'ex situ'
        ex_situ_count += 1

        if search(string2, text):
            list[3] = 'cleft'
        elif text.endswith('comment ?') or text.endswith('comment ça ?'):
            list[2] = 'wh'
            list[3] = 'none'
            list[4] = 'fragment'
            fragment_count += 1
        elif re.search('^comment\sje', text, flags=0) or re.search('^comment\stu', text, flags=0) or re.search('^comment\selle', text, flags=0) or re.search('^comment\sil', text, flags=0) or re.search('^comment\snous', text, flags=0) or re.search('^comment\svous', text, flags=0) or re.search('^comment\selles', text, flags=0) or re.search('^comment\sils', text, flags=0):
            list[3] = 'SV'
            sv_count += 1

    elif text.endswith('comment ?'):
        list[2] = 'in situ'
        list[3] = 'SV'
        list[4] = 'final'
        in_situ_count += 1

print(f'{string_count} occurrences of \'est-ce que\' found.')
print(f'{string2_count} occurrences of \'c\'est\' found.')
print(f'{ex_situ_count} occurrences of \'comment\' ex situ found.')
print(f'{in_situ_count} occurrences of \'comment\' in situ found.')
print(f'{fragment_count} occurrences of fragments found.')
print(f'{sv_count} occurrences of SV ordering found.')

We will now create a new .xlsx file with all the classified data.

In [ ]:
# creates spreadsheet using each nested list as a row

df = pd.DataFrame(data=all_rows)

# converts into excel
df.to_excel("all_rows_classified.xlsx", index=False)

print("Dictionary converted into excel...")

In [ ]:
# opens the new excel sheet and reads it
# I created a .xls copy of the .xlsx file manually otherwise it won't open

file2 = 'all_rows_classified.xls'
data2 = pd.ExcelFile(file2)

# returns the all the sheet names within the excel file
print(data2.sheet_names)

In [ ]:
# parses the sheet into a data frame to show the column structure within the file

data_frame2 = data2.parse('Sheet1')
data_frame2.info

# only shows the first 10 rows
data_frame2.head(60)